In [1]:
import telebot
bot = telebot.TeleBot(<API-KEY>, parse_mode='MARKDOWN')

In [2]:
import account_manager

In [3]:
import message_analyzer

In [4]:
@bot.message_handler(commands=['start'])
def check_user_account(message):
    bot.send_message(message.from_user.id, message.from_user.first_name + ', '+ account_manager.Account(message.from_user.id).check())

In [5]:
@bot.message_handler(content_types=['text'])
def just_do_it(message):
    analyzed = message_analyzer.Analyzer().message_analyzer(message.text.lower())
    output_message = '\n'.join(analyzed[0])
    try: # может придти ValueError
        output_data = analyzed[1][0] 
    except:
        output_data= []
    output_code = analyzed[2]
    if 'add' in output_code:
        bot.send_message(message.from_user.id, output_message + '\n\n' + account_manager.Account(message.from_user.id).add_expense(output_data))
    elif len(set(output_code) & set(['get_all', 'get_date', 'get_category'])) > 0:
        bot.send_message(message.from_user.id, output_message + '\n\n' + account_manager.Account(message.from_user.id).get_expense(output_code))
    elif 'get_account' in output_code:
        bot.send_message(message.from_user.id, output_message + '\n\n' + account_manager.Account(message.from_user.id).get_all_expense(output_code))
    else:
        bot.send_message(message.from_user.id, output_message)    

In [ ]:
bot.polling(none_stop=True, interval=0)